In [1]:
%pip install -qU langchain langchain-core langchain-community langchain-openai langchain-anthropic langchainhub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


%load_ext autoreload
%autoreload 2


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.chat_models import init_chat_model

llm = init_chat_model('gpt-4o-mini', temperature=0)



c:\Users\samue\AppData\Local\pypoetry\Cache\virtualenvs\sociaty-newsletter-generator-brUSjMkx-py3.12\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `init_chat_model` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [4]:
from sociaty_newsletter_generator.models import (
    Cluster,
    ClusteringSession,
    SetOfUniqueArticles,
    Article,
)

# Database utilities


In [5]:
from sociaty_newsletter_generator.db import init_db

await init_db()

In [6]:
sessions = await ClusteringSession.find_many().to_list()

for session in sessions:
    print(session.id)
    
    start, end = session.data_start, session.data_end
    nb_days = (end - start).days
    print(f" - {start.strftime('%Y-%m-%d')} -> {end.strftime('%Y-%m-%d')} ({nb_days} days)")
    print(f" - {session.clusters_count} total clusters")

    included_clusters = await session.get_included_sorted_clusters()
    print(f" - {len(included_clusters)} included clusters")
    
    print(" - Example of good clusters")
    for cluster in included_clusters[:10]:
        print(f"    - {cluster.title}")

    print()



6697a91d6720e5b25ef7946b
 - 2024-06-29 -> 2024-07-02 (3 days)
 - 109 total clusters
 - 73 included clusters
 - Example of good clusters
    - SK Hynix's $75 Billion Investment Focuses on AI-Enhanced HBM Processors
    - Controversy Surrounding AI Training on Web Content Sparks Legal Battles
    - Morgan Freeman Condemns Unauthorized AI Voice Imitations
    - Apple Working to Integrate AI Features into Vision Pro Headsets
    - States Addressing AI Skills Gap in Workforce
    - Apple Plans to Monetize Advanced AI Features with Subscription Model
    - Apple to Integrate Google's Gemini AI into iPhone for Enhanced AI Capabilities
    - Chinese AI Startups Flock to Singapore for Global Expansion Amid US Restrictions
    - Top Artificial Intelligence (AI) Stocks for Long-Term Investment
    - Robinhood Enhances Investing App with AI Tools Through Acquisition of Pluto Capital

6697ab4a6720e5b25ef794d9
 - 2024-07-02 -> 2024-07-05 (3 days)
 - 264 total clusters
 - 152 included clusters
 - Exa

# Summarize one subject


In [7]:
session_id ="669a345da02feb1858343077"
session = await ClusteringSession.get(session_id)
assert session

clusters = await session.get_included_sorted_clusters()

first_cluster = clusters[0]
print(f"{first_cluster.title} ({first_cluster.articles_count} articles)")
print(first_cluster.summary)

articles = SetOfUniqueArticles(await first_cluster.get_articles())

def format_articles(articles: SetOfUniqueArticles):
    return "\n\n".join(
        f"{article.title}\n{article.date}\n{article.url}\n{article.body}"
       for article in articles
    )

print(format_articles(articles))

OpenAI Introduces GPT-4o Mini: A Cheaper and More Accessible AI Model (71 articles)
OpenAI has unveiled GPT-4o mini, a smaller and more affordable AI model that offers advanced intelligence capabilities at a lower cost. This new model is 60% cheaper than previous versions, making it accessible to a wider range of developers and users.
OpenAI unveils GPT-4o mini, a smaller and cheaper AI model
2024-07-18 15:34:00
https://techcrunch.com/2024/07/18/openai-unveils-gpt-4o-mini-a-small-ai-model-powering-chatgpt/
OpenAI introduced GPT-4o mini on Thursday, its latest small AI model. The company says GPT-4o mini, which is cheaper and faster than OpenAI's current

ChatGPT Maker OpenAI Goes Smaller and Cheaper with New AI Tech
2024-07-18 16:58:00
https://www.wsj.com/tech/ai/chatgpt-maker-goes-smaller-and-cheaper-with-new-ai-tech-0cbcff84?mod=tech_lead_pos1
OpenAI released a smaller and cheaper version of the technology that powers ChatGPT, part of an industry shift toward more efficient artificia

In [8]:
from typing_extensions import Literal
from langchain.chat_models.base import BaseChatModel

In [9]:
sociaty_newsletter_template = """ # **SocIAty Intelligence - 24 Juillet 2024*

*Votre source d'insights sur l'intelligence artificielle*

<introduction>

---

## 🚀 L'IA en Action
<!-- Commentaire : cette section contient les sujets principaux. Il y a deux sujets à titre d'exemple, mais ça peut être plus. -->

### <sujet1_titre>
![sujet1_image](sujet1_image_url) <!-- Remplacer par l'URL de l'image -->

**<sujet1_resumé>[Lorem ipsum dolor sit amet](https://lien_vers_la_source_officielle), consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. </sujet1_résumé>**

<sujet1_détails>

- Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo.
- Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit, sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem.
- Ut enim ad minima veniam, quis nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur?
- Quis autem vel eum iure reprehenderit qui in ea voluptate velit esse quam nihil molestiae consequatur, vel illum qui dolorem eum fugiat quo voluptas nulla pariatur?

</sujet1_détails>

### <sujet2_titre>
![sujet2_image](sujet1_image_url) <!-- Remplacer par l'URL de l'image -->

**<sujet2_resumé>[Lorem ipsum dolor sit amet](https://lien_vers_la_source_officielle2), consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. </sujet2_résumé>**

<sujet2_détails>

- Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo.
- Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit, sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem.
- Ut enim ad minima veniam, quis nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur?
- Quis autem vel eum iure reprehenderit qui in ea voluptate velit esse quam nihil molestiae consequatur, vel illum qui dolorem eum fugiat quo voluptas nulla pariatur?

</sujet2_détails>

<!-- Commentaire : S'il y en a, insérer ici les autres sujets principaux, sous le même format-->


## 📰 Actualités et Tendances
<!-- Commentaire : cette section contient les sujets secondaires. Il y a trois sujets à titre d'exemple, mais ça peut être plus. -->

**XXX** a [annoncé](https://lien_vers_la_source_officielle) ipsum dolor sit amet, consectetur adipiscing elit. Etiam venenatis felis id ligula porttitor sollicitudin. Nulla facilisi. Morbi commodo arcu eu varius condimentum. Aenean felis eros, pharetra quis mi eu, sagittis dapibus ante. 

**YYY** [s’inquiète](https://lien_vers_la_source_officielle) Duis finibus dolor elementum ex rhoncus interdum. Aliquam tincidunt rhoncus bibendum. Fusce scelerisque est vel tempus volutpat. Sed nec auctor est, in sollicitudin ante. Nunc viverra, quam in pharetra gravida, orci tellus auctor elit, et tincidunt turpis augue in nisi. Integer ac nulla id urna pulvinar imperdiet.

**ZZZ** [verbe](https:///lien_vers_la_source_officielle)  usce tempus faucibus vestibulum. Phasellus sem mi, facilisis pulvinar nisl a, consectetur posuere turpis. Curabitur commodo ante eu sapien tristique, sit amet ultricies diam convallis. Nulla nec ligula consectetur, imperdiet ipsum non, rutrum urna. Vivamus imperdiet sagittis mi vel dapibus.

</autres_sujets_secondaires>

---

<phrase_de_conclusion>

https://www.linkedin.com/company/sociaty-io/

Nos offres https://sociaty.io/#offres

Contactez-nous https://sociaty.io/#projet
"""

In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from textwrap import dedent

gpt4o = init_chat_model('gpt-4o')

prompt_template = PromptTemplate.from_template(dedent("""You are a a newsletter writer.  

Your task is to fill in the template below with the latest news and trends in the field of artificial intelligence.
                                                      
Here is the template : 
<newsletter_template>
{newsletter_template}
</newsletter_template>

Here are the main topics :
<main_topics>
{main_topics}
</main_topics>

Here are the secondary topics : 
<secondary_topics>
{secondary_topics}
</secondary_topics>

All the topics must be included in the newsletter even if the template 

You have to write the newsletter in the {language} language.
The audience is familiar with the topic.
Newsletter writing date : {writing_date}
We consider what happened from {data_start} to {data_end}.
"""
)
)


chain = prompt_template|gpt4o | StrOutputParser()

In [11]:
from pydantic import BaseModel, Field, HttpUrl

class Topic(BaseModel):
    comprehensive_summary:str
    url:HttpUrl
    image_url:HttpUrl| None = None

    def __str__(self):
        t = f"{self.url}\n{self.comprehensive_summary}"
        if self.image_url:
            t += f"\nImage url : {self.image_url}"
        return t

In [12]:
from sociaty_newsletter_generator.services import create_articles_summarizer_chain, Language

summarizer_chain = create_articles_summarizer_chain(language='fr')

async def cluster_to_topic(cluster:Cluster, 
                             summarizer : ArticlesSummarizer,
                             )->Topic:

    articles = SetOfUniqueArticles(await cluster.get_articles())

    summary = summarizer_chain.invoke(articles)

    # get first not none image from articles
    image_url = next((article.image for article in articles if article.image), None)

    return Topic(comprehensive_summary=summary.final_summary,
                   url=articles[0].url,
                     image_url=image_url)

In [12]:
language = 'fr'

summarizer = ArticlesSummarizer()
main_clusters = clusters[:5]
secondary_clusters = clusters[5:10]

main_topics = [await cluster_to_topic(cluster, language=language, llm=llm) for cluster in main_clusters]
secondary_topics = [await cluster_to_topic(cluster, language=language, llm=llm) for cluster in secondary_clusters]

TypeError: cluster_to_topic() got an unexpected keyword argument 'language'

In [34]:
from datetime import date

main_topics_str = "\n\n".join([
    f"<main_topic_{i}>\n{str(topic)}\n</main_topic_{i}>" for i, topic in enumerate(main_topics,1)
])

secondary_topics_str = "\n\n".join([
    f"<secondary_topic_{i}>\n{str(topic)}\n</secondary_topic_{i}>" for i, topic in enumerate(secondary_topics,1)
])

In [37]:
assert session

newsletter = chain.invoke({'newsletter_template':sociaty_newsletter_template,
                'main_topics':main_topics_str,
                'secondary_topics':secondary_topics_str,
                'language':language,
                'writing_date': date.today().strftime("%d %B %Y"),
                'data_start': session.data_start.strftime("%d %B %Y"),
                'data_end': session.data_end.strftime("%d %B %Y"),
                 })

print(newsletter)

<newsletter_template>
 # **SocIAty Intelligence - 01 Août 2024*

*Votre source d'insights sur l'intelligence artificielle*

Bienvenue dans cette édition de SocIAty Intelligence, votre guide incontournable sur les dernières nouvelles et tendances en intelligence artificielle. Du lancement de nouveaux modèles révolutionnaires aux défis réglementaires, plongez avec nous dans l'actualité IA la plus récente.

---

## 🚀 L'IA en Action

### OpenAI dévoile GPT-4o Mini : Un modèle compact et performant
![OpenAI GPT-4o Mini](https://techcrunch.com/wp-content/uploads/2024/05/openAI-spiral-teal.jpg?resize=1200,675)

**[OpenAI a lancé le GPT-4o mini](https://techcrunch.com/2024/07/18/openai-unveils-gpt-4o-mini-a-small-ai-model-powering-chatgpt/), un modèle d'IA plus petit, moins cher et plus rapide que ses prédécesseurs, notamment le GPT-3.5 Turbo.**

Ce modèle est conçu pour être accessible aux développeurs et aux utilisateurs, avec un coût de 15 cents par million de jetons d'entrée et 60 cents pa

# Langchain


In [35]:
from textwrap import dedent
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import Runnable, RunnableLambda, RunnablePassthrough
from typing import Literal, Sequence
from langsmith import traceable
from pydantic import BaseModel, Field
from langchain.chat_models.base import BaseChatModel
from langchain.chat_models import init_chat_model

from sociaty_newsletter_generator.models import Article, SetOfUniqueArticles


{'title': 'RunnableParallel<formatted_articles,language>Input',
 'type': 'object',
 'properties': {}}

In [58]:
Language = Literal["fr", "en"]

class SummaryOutput(BaseModel):
    """Output for the summarization process. Every field is mandatory."""

    scratchpad: str = Field(
        ...,
        description="Very dense set of facts and ideas separated by comas that should cover a large amount of the content.",
    )
    missing_entities: str = Field(
        ...,
        description="Mandatory list of ideas or facts that you forgot to include in the scratchpad.",
    )
    final_summary: str = Field(
        ...,
        description="Final comprehensive summary of the content including the scratchpad and forgot facts.",
    )


type ArticlesSummarizerChain = Runnable[SetOfUniqueArticles, str]

def create_articles_summarizer_chain(
        llm: BaseChatModel | str = "gpt-4o-mini",
        language: Language = "en",
        articles_limit: int | None = 100,
    ) -> ArticlesSummarizerChain:
        if isinstance(llm, str):
            llm = init_chat_model(llm)
        structured_llm = llm.with_structured_output(SummaryOutput)

        def _format_articles(articles: SetOfUniqueArticles):
            return "\n\n".join(
                f"{article.title}\n{article.date}\n{article.url}\n{article.body}"
                for article in articles.limit(articles_limit)
            )
        
        prompt = PromptTemplate.from_template(
            dedent("""Here is a set of news articles to summarize : 
            <articles>
            {formatted_articles}
            </articles>

            Write a comprehensive summary of this content in {language} language.
            Include as much facts and ideas as possible in the summary.
        """)
        )

        return {
            "formatted_articles": RunnableLambda(lambda x: _format_articles(x)),
            "language" : RunnableLambda(lambda _: language)
        }| prompt| structured_llm | RunnableLambda(lambda x: x.final_summary)




chain = create_articles_summarizer_chain(language='fr')

chain.invoke(articles)


"Le 18 juillet 2024, OpenAI a lancé le modèle GPT-4o mini, une version réduite et économique de son modèle phare GPT-4o, visant à rendre l'intelligence artificielle plus accessible. Ce modèle est 60 % moins cher que le précédent, GPT-3.5 Turbo, avec un coût de 15 cents par million de jetons d'entrée et 60 cents par million de jetons de sortie. GPT-4o mini est également plus rapide et nécessite moins de puissance de calcul, ce qui le rend adapté aux petites entreprises et aux développeurs à budget limité. Il surpasse ses rivaux comme Gemini 1.5 Flash de Google et Claude 3 Haiku d'Anthropic dans divers benchmarks, y compris le raisonnement mathématique et multimodal. Le modèle prend en charge des entrées et sorties textuelles et visuelles, avec des fonctionnalités multimodales à venir. Il remplace officiellement GPT-3.5 dans ChatGPT et est disponible pour les utilisateurs gratuits et payants. OpenAI vise à élargir son marché et à attirer de nouveaux clients grâce à cette initiative, tout

In [60]:
clusters = await session.get_included_sorted_clusters()

main_clusters = clusters[:100]

sets_of_articles = [SetOfUniqueArticles(await cluster.get_articles()) for cluster in main_clusters]

chain = create_articles_summarizer_chain(language='fr', articles_limit=5)
# chain.batch(sets_of_articles)


["OpenAI a lancé le 18 juillet 2024 le GPT-4o mini, un modèle d'IA plus petit et moins cher, qui est plus rapide et économique par rapport à ChatGPT et à d'autres modèles plus grands. Ce modèle, le plus abordable de la gamme d'OpenAI, est annoncé comme étant 60% moins cher que le GPT-3.5 Turbo et surpasse les modèles Gemini 1.5 Flash et Claude 3 Haiku selon le benchmark MMLU. Cette initiative s'inscrit dans une tendance de l'industrie vers des modèles d'IA plus efficaces, rendant l'intelligence avancée plus accessible aux développeurs et utilisateurs. Les modèles plus petits nécessitent moins de puissance de calcul, ce qui les rend rentables et facilement évolutifs, permettant ainsi aux petits développeurs, qui n'ont pas nécessairement de gros budgets, d'exploiter l'IA générative pour développer leurs applications.",
 "Meta a annoncé qu'il ne lancera pas son modèle multimodal Llama AI dans l'Union européenne à cause de l'environnement réglementaire imprévisible et des protections de do